In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
item_c= pd.read_csv('./data/item_categories.csv') # 아이템 카테고리 / ID c : item_category_name, item_category_id
item= pd.read_csv('./data/items.csv') # 카테고리, 아이디, 이름 c: item_name, item_id, item_category_id
train_sale= pd.read_csv('./data/sales_train.csv')  # train data c : data, date_block_num, shop_id,item_id,item_price,item_cnt_day
# item_cnt_day를 예측
sample_sub = pd.read_csv('./data/sample_submission.csv') # 세일하는 기간 ID : 모두 0.5달 세일 c : ID, item_cnt_month
shop= pd.read_csv('./data/shops.csv') # 상점 이름, ID c : show_name,show_id
test= pd.read_csv('./data/test.csv') # 각 상점마다 세일을 어떻게 하는지, 어떤 품목을 세일하는지 c : ID, shop_id,item_id

In [30]:
item_c.info(), item.info(), sales_tr.info(), sample_sub.info(), shop.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item

(None, None, None, None, None, None)

In [7]:
train_sale[train_sale.duplicated()]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
76962,05.01.2013,0,54,20130,149.0,1.0
1435367,23.02.2014,13,50,3423,999.0,1.0
1496766,23.03.2014,14,21,3423,999.0,1.0
1671873,01.05.2014,16,50,3423,999.0,1.0
1866340,12.07.2014,18,25,3423,999.0,1.0
2198566,31.12.2014,23,42,21619,499.0,1.0


In [11]:
train_sale[train_sale['item_cnt_day'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2,05.01.2013,0,25,2552,899.0,-1.0
148,23.01.2013,0,25,2321,999.0,-1.0
175,07.01.2013,0,25,2199,1449.0,-1.0
807,02.01.2013,0,25,2330,599.0,-1.0
1041,13.01.2013,0,25,5034,1989.0,-1.0
...,...,...,...,...,...,...
2934243,26.10.2015,33,25,3917,449.0,-1.0
2934462,18.10.2015,33,25,4896,6398.0,-1.0
2935263,05.10.2015,33,25,10039,249.0,-1.0
2935643,16.10.2015,33,25,7893,2990.0,-1.0


In [17]:
(train_sale['item_cnt_day'] < 0).sum()

7356

In [61]:
train_sale.value_counts_counts()

date        date_block_num  shop_id  item_id  item_price  item_cnt_day
23.02.2014  13              50       3423     999.0       1.0             2
01.05.2014  16              50       3423     999.0       1.0             2
05.01.2013  0               54       20130    149.0       1.0             2
23.03.2014  14              21       3423     999.0       1.0             2
31.12.2014  23              42       21619    499.0       1.0             2
                                                                         ..
11.01.2015  24              27       3077     799.0       1.0             1
                                     3335     1499.0      1.0             1
                                     3340     1799.0      1.0             1
                                     3392     499.5       1.0             1
31.12.2014  23              59       22087    99.0        1.0             1
Length: 2935843, dtype: int64

In [29]:
train_sale[train_sale.item_cnt_day > 350].shop_id

1501160    24
1573252    27
1573253    27
1708207    25
2048518    12
2067667    55
2067669    55
2067677    55
2257299    12
2326930    12
2608040    12
2625847    12
2626181    12
2851073    55
2851091    55
2864235    12
2864260    12
2909401    12
2909818    12
Name: shop_id, dtype: int64

In [32]:
item[item.item_id == 24]

,item_name,item_id,item_category_id
24,/СМЕРТЕЛЬНЫЙ РАСКЛАД,24,40


In [73]:
train_sale['item_cnt_day'].unique()

array([ 1.000e+00, -1.000e+00,  3.000e+00,  2.000e+00,  4.000e+00,
        5.000e+00,  1.300e+01,  7.000e+00,  6.000e+00,  8.000e+00,
        1.000e+01,  1.100e+01,  3.000e+01,  2.200e+01,  1.400e+01,
        1.500e+01,  9.000e+00,  1.200e+01,  1.700e+01,  1.900e+01,
        2.000e+01, -5.000e+00, -6.000e+00, -3.000e+00, -2.000e+00,
        1.600e+01,  6.400e+01,  1.800e+01,  2.900e+01,  2.600e+01,
        3.900e+01,  2.100e+01,  2.700e+01,  5.300e+01,  2.500e+01,
        2.300e+01,  2.400e+01,  4.000e+01,  2.800e+01,  3.100e+01,
        3.600e+01, -4.000e+00,  3.200e+01,  3.500e+01,  4.100e+01,
        8.800e+01,  4.400e+01,  5.200e+01,  5.600e+01,  3.800e+01,
        3.400e+01,  5.700e+01,  6.900e+01,  3.700e+01,  4.700e+01,
        5.000e+01,  4.800e+01,  3.300e+01,  6.100e+01,  8.000e+01,
        4.200e+01,  8.200e+01,  2.000e+02,  1.000e+02,  7.600e+01,
        7.700e+01,  1.070e+02,  4.600e+01,  4.900e+01,  7.000e+01,
        5.500e+01,  6.000e+01,  4.500e+01,  7.100e+01,  6.500e

## Data Description

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.
### File descriptions
- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
- sample_submission.csv - a sample submission file in the correct format.
- items.csv - supplemental information about the items/products.
- item_categories.csv  - supplemental information about the items categories.
- shops.csv- supplemental information about the shops.
## Data fields
- ID - an Id that represents a (Shop, Item) tuple within the test set
- shop_id - unique identifier of a shop
- item_id - unique identifier of a product
- item_category_id - unique identifier of item category
- **item_cnt_day - number of products sold. You are predicting a monthly amount of this measure**
- item_price - current price of an item
- date - date in format dd/mm/yyyy
- **date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33**
- item_name - name of item
- shop_name - name of shop
- item_category_name - name of item category

In [33]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [11]:
a = item_c.merge(item).merge(sales_tr)

In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


In [37]:
sample_sub.value_counts()

ID      item_cnt_month
0       0.5               1
142765  0.5               1
142793  0.5               1
142794  0.5               1
142795  0.5               1
                         ..
71403   0.5               1
71404   0.5               1
71405   0.5               1
71406   0.5               1
214199  0.5               1
Length: 214200, dtype: int64

In [22]:
sales_tr[~sales_tr.isnull()]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [66]:
item_c

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [67]:
item

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [68]:
train_sale

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [69]:
sample_sub 

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [70]:
shop

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [71]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [2]:
sales

NameError: name 'sales' is not defined

In [1]:
q = sales.sort_values('date')
w = sales.sort_values('date_num_block')

NameError: name 'sales' is not defined